In [1]:
# Current directory
import os
os.chdir('F:/One/ACP')

### Fusion models

In [2]:
import numpy as np
import pandas as pd


# train
train_features = pd.read_csv("features_label/train.csv", index_col=False, header=None)
train_labels = pd.read_csv("features_label/train_label.csv", index_col=False, header=None)
# test
test_features = pd.read_csv("features_label/test.csv", index_col=False, header=None)
test_labels = pd.read_csv("features_label/test_label.csv", index_col=False, header=None)

print('train_features:', train_features.shape)
print('train_labels', train_labels.shape)
print('test_features', test_features.shape)
print('test_labels', test_labels.shape)


train_features = np.array(train_features)
train_labels = np.array(train_labels)
test_features = np.array(test_features)
test_labels = np.array(test_labels)


train_features: (1378, 320)
train_labels (1378, 1)
test_features (344, 320)
test_labels (344, 1)


In [4]:
import math
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc


# Create a directory to save the model
model_GRU_dir = 'save_models/GRU/Independence'
model_CNN_dir = 'save_models/CNN/Independence'
model_CapsuleGAN_dir = 'save_models/CapsuleGAN/Independence'

[sample_num, input_dim] = np.shape(train_features)
y_train = train_labels
y_test = test_labels
out_dim=2

ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
AP=[]

all_predictions = []
for i in range(10):
    # Loading model
    if i == 0:
        X_train = np.reshape(train_features, (-1,1,input_dim))
        X_test = np.reshape(test_features, (-1,1,input_dim))

        model_path = os.path.join(model_GRU_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')
    elif i in (1, 2, 3, 4, 5, 6, 7):
        X_train = np.reshape(train_features, (-1,1,input_dim))
        X_test = np.reshape(test_features, (-1,1,input_dim))

        model_path = os.path.join(model_CNN_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')
    else:
        X_train = train_features
        X_test = test_features

        model_path = os.path.join(model_CapsuleGAN_dir, f'ESM_{i}.h5')
        clf = load_model(model_path)
        print(f'Model for fold {i} loaded from {model_path}')

    y_score = clf.predict(X_test)
    all_predictions.append(y_score)


# 转换为 numpy 数组
all_predictions = np.array(all_predictions)

# 平均投票
average_predictions = np.mean(all_predictions, axis=0)
final_predictions = np.argmax(average_predictions, axis=1)

TP, FP, FN, TN = confusion_matrix(y_test, final_predictions).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
Sn_collecton = TP/(TP+FN)
Sp_collecton = TN/(TN+FP)
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton = MCC
BACC_collecton = 0.5*TP/(TP+FN)+0.5*TN/(TN+FP)
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton = ACC
# ROC curve
fpr, tpr, _ = roc_curve(y_test, average_predictions[:, 1])
auc_roc = auc(fpr, tpr)
AUC_collecton = auc_roc
# PR curve
precision, recall, _ = precision_recall_curve(y_test, average_predictions[:, 1])
average_precision = average_precision_score(y_test, average_predictions[:, 1])
AP = average_precision


# Output
results = [
    f"ACC: {round(ACC_collecton, 3)}",
    f"BACC: {round(BACC_collecton, 3)}",
    f"Sn: {round(Sn_collecton, 3)}",
    f"Sp: {round(Sp_collecton, 3)}",
    f"MCC: {round(MCC_collecton, 3)}",
    f"AUC: {round(AUC_collecton, 3)}",
    f"AP: {round(AP, 3)}"
]

for result in results:
    print(result)

# Append the results to the file
with open('result/results_Vote.txt', 'a') as file:
    file.write("----------------------------------------\n")
    for result in results:
        file.write(result + '\n')


Model for fold 0 loaded from save_models/GRU/Independence\ESM_0.h5
Model for fold 1 loaded from save_models/CNN/Independence\ESM_1.h5
Model for fold 2 loaded from save_models/CNN/Independence\ESM_2.h5
Model for fold 3 loaded from save_models/CNN/Independence\ESM_3.h5
Model for fold 4 loaded from save_models/CNN/Independence\ESM_4.h5
Model for fold 5 loaded from save_models/CNN/Independence\ESM_5.h5
Model for fold 6 loaded from save_models/CNN/Independence\ESM_6.h5
Model for fold 7 loaded from save_models/CNN/Independence\ESM_7.h5
Model for fold 8 loaded from save_models/CapsuleGAN/Independence\ESM_8.h5
Model for fold 9 loaded from save_models/CapsuleGAN/Independence\ESM_9.h5
ACC: 0.779
BACC: 0.779
Sn: 0.786
Sp: 0.773
MCC: 0.558
AUC: 0.833
AP: 0.852


In [5]:
# 保存ROC曲线相关参数
np.savez('graph/ROC/Vote.npz', fpr=fpr, tpr=tpr, roc_auc=AUC_collecton)

# 保存PR曲线相关参数
np.savez('graph/PR/Vote.npz', recall=recall, precision=precision, average_precision=AP)

### end